# Text Generation with Neural Networks

In this notebook we will create a network that can generate text, here we show it being done character by character. Very awesome write up on this here: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

We organized the process into "steps" so you can easily follow along with your own data sets.

In [ ]:
# GOOGLE COLLAB USERS ONLY

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

## 1. Read Data

You can grab any free text you want from here: https://www.gutenberg.org/

We'll choose all of shakespeare's works (which we have already downloaded for you), mainly for two reasons:

1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.


2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

In [2]:
path_to_file = '/content/shakespeare.txt'

In [3]:
text = open(path_to_file, mode='r').read()

In [4]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


## <a> Unique Characters

In [5]:
vocab = sorted(set(text))

In [6]:
print(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|']


---------------
--------
## 2. Text Preprocessing

## <a>* Text Vectorization
    We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

### <i><a>- Character to index

In [7]:
char_to_index = {char : index for index, char in enumerate(vocab)}

In [8]:
char_to_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82}

In [9]:
len(char_to_index)

83

In [10]:
char_to_index['y']

80

### <i><a> - index to char

In [11]:
index_to_char = np.array(vocab)

In [12]:
index_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|'], dtype='<U1')

In [13]:
len(index_to_char)

83

In [14]:
index_to_char[25]

'?'

### <i><a> - Encoding the character of the whole text

In [15]:
encoding_text = np.array([char_to_index[c] for c in text])

In [16]:
encoding_text

array([ 0,  1,  1, ...,  1,  1, 39])

In [17]:
encoding_text.shape

(3145728,)

In [18]:
encoding_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

### <i><a> Mapping

In [19]:
sample = text[:20]
sample

'\n                   '

------
------
## 3. Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

###  <i><a> First we need understanding the text we are using

In [20]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [21]:
line = 'From fairest creatures we desire increase,'
len(line)

42

In [22]:
lines = '''
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
'''
len(lines)

135

### <i><a> Traning Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam" Sequence Out: "ello my name"

We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [23]:
sequence_length = 120

In [24]:
# calculate how many sequence for the whole text,because of 0 indeing we will add 1
total_num_sequence = len(text) // (sequence_length + 1)

In [25]:
total_num_sequence

25997

### <i><a> Create Training Sequences (encoding_text)

In [26]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoding_text)

In [27]:
type(char_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [28]:
# create training sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoding_text)

for item in char_dataset.take(500):
    print(index_to_char[item.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing whether the last batch should be dropped in the case it has fewer than `batch_size` elements; the default behavior is not to drop the smaller batch.

### <i><a> Creating Sequence Batches

In [29]:
sequence = char_dataset.batch(sequence_length+1, drop_remainder=True)

In [30]:
sequence

<_BatchDataset element_spec=TensorSpec(shape=(121,), dtype=tf.int64, name=None)>

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [31]:
def create_sequence_target(seq):
    input_text = seq[:-1] # hello my nam
    target_text = seq[1:] # ello my name
    return(input_text, target_text)

In [32]:
dataset = sequence.map(create_sequence_target)
dataset

<_MapDataset element_spec=(TensorSpec(shape=(120,), dtype=tf.int64, name=None), TensorSpec(shape=(120,), dtype=tf.int64, name=None))>

In [33]:
for input_text, target_text in dataset.take(1):
    print(input_text.numpy())
    print(''.join(index_to_char[input_text.numpy()]))

    print('\n')

    print(target_text.numpy())
    print(''.join(index_to_char[target_text.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### <i><a> Generating Training Batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text

In [34]:
# batch_size
batch_size = 128

buffer_size = 1000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [35]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

In [36]:
512*2


1024

------
-----
## 4. Creating The Model

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and two LSTM layers. We based this model architecture off the <a href="https://deepmoji.mit.edu/">DeepMoji</a> and the original source code can be found <a href ='https://github.com/bfelbo/DeepMoji'>here</a>.

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [37]:
# length of the vocabulary in char
vocab_size = len(vocab)

# the embeding dimension
embed_dim = 1000

# numbers of RNN units
rnn_neurons  = 1026

In [38]:
vocab_size

83

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, GRU

### <i><a> Setting up loss function

Now let's create a function that easily adapts to different variables as shown above.

In [40]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [41]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module keras.src.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1, ignore_class=None)
    Computes the sparse categorical crossentropy loss.
    
    Standalone usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    >>> y_true = [[[ 0,  2],
    ...            [-1, -1]],
    ...           [[ 0,  2],
    ...            [-1, -1]]]
    >>> y_pred = [[[[1.0, 0.0, 0.0], [0.0, 0.0, 1.0]],
    ...             [[0.2, 0.5, 0.3], [0.0, 1.0, 0.0]]],
    ...           [[[1.0, 0.0, 0.0], [0.0, 0.5, 0.5]],
    ...            [[0.2, 0.5, 0.3], [0.0, 1.0, 0.0]]]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(
    ...   y_true, y_pred, ignore_class=-1)
    >>> loss.numpy()
    array([[[2

In [42]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [43]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()

    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))

    model.add(GRU(rnn_neurons,
                          return_sequences=True,
                          stateful=True,
                          recurrent_initializer='glorot_uniform'))

    model.add(Dense(vocab_size))

    model.compile(optimizer='adam', loss=sparse_cat_loss)

    return model

In [44]:
model = create_model(vocab_size = vocab_size,
                     embed_dim = embed_dim,
                     rnn_neurons = rnn_neurons,
                     batch_size  = batch_size
                    )

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 1000)         83000     
                                                                 
 gru (GRU)                   (128, None, 1026)         6242184   
                                                                 
 dense (Dense)               (128, None, 83)           85241     
                                                                 
Total params: 6410425 (24.45 MB)
Trainable params: 6410425 (24.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


-------
------
## 5. Training the Model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.

In [46]:
for input_example_batch, target_example_batch in dataset.take(1):

    # predict off some random batch
    example_batch_prediction = model(input_example_batch)

    # Display off dimentions of the prediction
    print(example_batch_prediction.shape, "(batch_size, sequence_length, vocab_size)")

(128, 120, 83) (batch_size, sequence_length, vocab_size)


In [47]:
example_batch_prediction

<tf.Tensor: shape=(128, 120, 83), dtype=float32, numpy=
array([[[-2.24607605e-02,  1.62957178e-03, -4.22578352e-03, ...,
          1.28339359e-03,  1.00502875e-02,  2.78951172e-02],
        [-1.04436381e-02,  1.35153988e-02,  1.83796939e-02, ...,
         -4.16764207e-02,  2.36079115e-02, -5.68935466e-05],
        [-7.86450133e-03,  2.66562365e-02,  6.50447747e-03, ...,
         -3.06669492e-02,  2.19769161e-02,  1.14161531e-02],
        ...,
        [-6.42879307e-03, -1.66745652e-02,  2.50323117e-02, ...,
         -1.47083448e-02, -3.85230698e-04,  1.42078698e-02],
        [-1.30182505e-02, -1.50342248e-02,  2.10862122e-02, ...,
         -2.48284154e-02, -1.68702658e-02,  2.02731285e-02],
        [-1.25386408e-02,  6.88271550e-03,  1.01287952e-02, ...,
         -2.06914730e-02,  1.86875034e-02,  3.92967202e-02]],

       [[-1.26646738e-02, -1.05183898e-02,  1.66758467e-02, ...,
          5.89675146e-05, -1.36138396e-02,  7.72566488e-03],
        [ 5.57324942e-03, -1.57681154e-03, -1.1

In [48]:
sampled_indices = tf.random.categorical(example_batch_prediction[0], num_samples =1 )

In [49]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[11],
       [13],
       [44],
       [59],
       [13],
       [61],
       [82],
       [56],
       [43],
       [21],
       [73],
       [29],
       [ 2],
       [15],
       [11],
       [71],
       [69],
       [ 7],
       [68],
       [22],
       [66],
       [38],
       [78],
       [25],
       [46],
       [ 2],
       [48],
       [18],
       [73],
       [ 9],
       [72],
       [12],
       [12],
       [60],
       [75],
       [23],
       [ 3],
       [30],
       [ 9],
       [57],
       [18],
       [ 6],
       [68],
       [81],
       [70],
       [57],
       [82],
       [58],
       [67],
       [51],
       [75],
       [62],
       [69],
       [45],
       [36],
       [13],
       [ 6],
       [35],
       [13],
       [69],
       [73],
       [34],
       [70],
       [32],
       [56],
       [48],
       [16],
       [82],
       [46],
       [55],
       [39],
       [37],
       [68],
   

In [50]:
# reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [51]:
sampled_indices

array([11, 13, 44, 59, 13, 61, 82, 56, 43, 21, 73, 29,  2, 15, 11, 71, 69,
        7, 68, 22, 66, 38, 78, 25, 46,  2, 48, 18, 73,  9, 72, 12, 12, 60,
       75, 23,  3, 30,  9, 57, 18,  6, 68, 81, 70, 57, 82, 58, 67, 51, 75,
       62, 69, 45, 36, 13,  6, 35, 13, 69, 73, 34, 70, 32, 56, 48, 16, 82,
       46, 55, 39, 37, 68,  9, 57, 49, 25, 36, 58, 20, 81, 36, 40,  4, 12,
       14, 13, 32, 19, 31, 71, 38, 56,  9,  2, 78,  5, 21, 16, 21, 80, 31,
       36, 61, 16, 69,  0, 72,  1, 47, 66, 59, 72, 65, 62, 71,  3, 12, 60,
       11])

In [52]:
print('Given the input seq :\n')
print(''.join(index_to_char[input_example_batch[0]]))

print('\n')
print('Next char Prediction : \n')
print(''.join(index_to_char[sampled_indices]))

Given the input seq :

ncing chips,
  O'er whom thy fingers walk with gentle gait,
  Making dead wood more blest than living lips,
    Since sa


Next char Prediction : 

02Sd2f|aR:rD!40pn)m;kMw?U!W7r-q11et<"E-b7(mzob|clZtgnTK2(J2nrIoGaW5|U`NLm-bX?Kc9zKO&132G8FpMa-!w':5:yFKf5n
q Vkdqjgp"1e0


In [53]:
epochs = 30

In [54]:
model.fit(dataset, epochs = epochs)

Epoch 1/30
203/203 [==============================] - 45s 201ms/step - loss: 2.2394
Epoch 2/30
203/203 [==============================] - 39s 193ms/step - loss: 1.6022
Epoch 3/30
203/203 [==============================] - 40s 198ms/step - loss: 1.4025
Epoch 4/30
203/203 [==============================] - 41s 202ms/step - loss: 1.3106
Epoch 5/30
203/203 [==============================] - 41s 202ms/step - loss: 1.2587
Epoch 6/30
203/203 [==============================] - 41s 202ms/step - loss: 1.2214
Epoch 7/30
203/203 [==============================] - 41s 199ms/step - loss: 1.1926
Epoch 8/30
203/203 [==============================] - 41s 202ms/step - loss: 1.1683
Epoch 9/30
203/203 [==============================] - 42s 204ms/step - loss: 1.1454
Epoch 10/30
203/203 [==============================] - 41s 198ms/step - loss: 1.1259
Epoch 11/30
203/203 [==============================] - 41s 202ms/step - loss: 1.1064
Epoch 12/30
203/203 [==============================] - 41s 200ms/step - lo

## Text Generation

In [55]:
model.save('shakespeare_gen.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [56]:
from tensorflow.keras.models import load_model

In [57]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size =1)

model.load_weights('/content/shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))

In [58]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 1000)           83000     
                                                                 
 gru_1 (GRU)                 (1, None, 1026)           6242184   
                                                                 
 dense_1 (Dense)             (1, None, 83)             85241     
                                                                 
Total params: 6410425 (24.45 MB)
Trainable params: 6410425 (24.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [59]:
def generate_text(model, start_seed, gen_size =1000, temp= 1.0):
    '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # number of charcters to generate
  num_generate = gen_size

  # vecotrizing starting seed text
  input_eval = [char_to_index[s] for s in start_seed]

  # expand to mach batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # empty list to hold resulting  to generated text
  text_generate = []


  temperature = temp

  # Here batch size ==1
  model.reset_states()

  for i in range(num_generate):

    # generate prediction
    prediction = model(input_eval)

    # remove the batch shape demention
    prediction = tf.squeeze(prediction, 0)

    # Use a categorical distribution to select the next character
    prediction = prediction/ temperature
    prediction_id = tf.random.categorical(prediction, num_samples = 1)[-1, 0].numpy()

    # pas the prediction character for the next input
    input_eval = tf.expand_dims([prediction_id], 0)

    # transform back to character letter
    text_generate.append(index_to_char[prediction_id])
  return (start_seed + ''.join(text_generate))


In [60]:
print(generate_text(model, 'love', gen_size =1000))

love, if
    despite remember.
  ESCALUS. Are you to know that?
  ESCALUS. Tome all friends kill'd.
  PLANTAs d like me quite, whose friend
    Deed in the flat to him.
  DUKE. He draw the
    conquerv.
  DUKE. The philosomorsent Mariant mouth ever will return  
    Unfold what wild bears  The faces of allegiance fall before his fear,
     And you and you it is in truth of a man; be his guilt
     To undergo- I assure yet learn to kno?
  THIRD WITCH. He went to see.        Exit
  ESCALUS. Followed Cassius, Bacchus Late. I' th' studies shall a
     was wont to fear. Where will the Duke would the world                                                   Enter SIR NATHANIEL in mortality
    To bear with youth where Cawdor doth example.
  ESCALUS. What news?
  PHOLUS. Pardons me on the Dube; the other
    Content. The marketplayes were hard; this man.
  MALCOLM. Perchance.
  ANGELO. Truly hable to say
    Hourself with thy biat- a gentle people
    That for their due.
  GENTLEWOMAN. Stand cl